In [2]:
def heuristic_distance(state):
    """
    Calculates the heuristic distance by optimally pairing volunteers and victims.
    Args:
        state (dict): A dictionary containing positions of volunteers and victims.
    Returns:
        float: The total heuristic distance.
    """
    distances = []
    volunteers = state["volunteers"]
    victims = state["victims"]

    for volunteer_id, volunteer_pos in volunteers.items():
        for victim_id, victim_pos in victims.items():
            distance = compute_distance(volunteer_pos, victim_pos)
            distances.append((volunteer_id, victim_id, distance))

    distances.sort(key=lambda x: x[2])
    assigned_victims = set()
    total_distance = 0

    for _, victim_id, distance in distances:
        if victim_id not in assigned_victims:
            total_distance += distance
            assigned_victims.add(victim_id)

    return total_distance


def compute_distance(pos1, pos2):
    """
    Computes the Manhattan distance between two points.
    Args:
        pos1 (tuple): (x, y) coordinates of the first point.
        pos2 (tuple): (x, y) coordinates of the second point.
    Returns:
        int: The Manhattan distance.
    """
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

In [3]:
def heuristic_action_selection(state):
    """
    Selects the best actions for all volunteers based on their positions and heuristic distances.
    Args:
        state (dict): A dictionary containing positions of volunteers and victims.
    Returns:
        dict: A dictionary with volunteer IDs as keys and their respective actions as values.
    """
    actions = ["up", "down", "left", "right"]
    best_actions = {}

    for volunteer in state["volunteers"]:
        min_distance = float("inf")
        best_action = None
        for action in actions:
            next_state = perform_action(state, volunteer, action)
            distance = heuristic_distance(next_state)
            if distance < min_distance:
                min_distance = distance
                best_action = action
        best_actions[volunteer] = best_action

    return best_actions


def perform_action(state, volunteer, action):
    """
    Simulates the next state after performing an action for a volunteer.
    Args:
        state (dict): The current state containing positions.
        volunteer (str): The ID of the volunteer.
        action (str): The action to simulate.
    Returns:
        dict: The updated state after the action.
    """
    new_state = state.copy()
    x, y = new_state["volunteers"][volunteer]

    if action == "up":
        new_state["volunteers"][volunteer] = (x, y + 1)
    elif action == "down":
        new_state["volunteers"][volunteer] = (x, y - 1)
    elif action == "left":
        new_state["volunteers"][volunteer] = (x - 1, y)
    elif action == "right":
        new_state["volunteers"][volunteer] = (x + 1, y)

    return new_state


In [4]:
import json

def resq_rescue_scheduling(state, max_iterations=100):
    """
    Main rescue scheduling function using independent action selection and Q-learning.
    Args:
        state (dict): Initial state of volunteers and victims.
        max_iterations (int): Maximum number of iterations to run.
    Returns:
        dict: Final state after rescue completion.
    """
    t = 0
    alpha = 0.1  # Learning rate
    gamma = 0.99  # Discount factor
    q_values = {}  # Initializing Q-values

    while t < max_iterations and not is_rescue_complete(state):
        current_state = json.dumps(state, sort_keys=True)

        actions = heuristic_action_selection(state)

        for volunteer, action in actions.items():
            state = perform_action(state, volunteer, action)

        rewards = compute_rewards(state)

        serialized_state = json.dumps(state, sort_keys=True)

        for volunteer in state["volunteers"]:
            for action in actions.values():
                q_values[(current_state, action)] = q_values.get((current_state, action), 0)
                q_values[(current_state, action)] = (1 - alpha) * q_values[(current_state, action)] + \
                                                    alpha * (rewards[volunteer] + gamma * max(
                                                        q_values.get((serialized_state, a), 0)
                                                        for a in ["up", "down", "left", "right"]
                                                    ))

        t += 1

    state["time"] = t
    return q_values


def compute_rewards(state):
    """
    Computes the rewards based on the proximity of volunteers to victims.
    Args:
        state (dict): Current state of volunteers and victims.
    Returns:
        dict: Rewards for each volunteer.
    """
    rewards = {}
    for volunteer, position in state["volunteers"].items():
        closest_victim_distance = min(compute_distance(position, victim) for victim in state["victims"].values())
        rewards[volunteer] = -closest_victim_distance  # Negative reward for distance
    return rewards


def is_rescue_complete(state):
    """
    Checks if all victims are rescued.
    Args:
        state (dict): Current state of volunteers and victims.
    Returns:
        bool: True if all victims are rescued, False otherwise.
    """
    return len(state["victims"]) == 0


In [ ]:
import random
import json

def initialize_state(grid_size, num_volunteers, num_victims):
    """
    Initializes the state with random positions for volunteers and victims.
    Args:
        grid_size (int): The size of the grid.
        num_volunteers (int): Number of volunteers.
        num_victims (int): Number of victims.
    Returns:
        dict: The initial state with volunteers and victims.
    """
    state = {
        "volunteers": {
            f"volunteer_{i}": (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
            for i in range(num_volunteers)
        },
        "victims": {
            f"victim_{i}": (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
            for i in range(num_victims)
        },
    }
    return state

if __name__ == "__main__":
    grid_size = 25
    num_volunteers = 5
    num_victims = 8

    state = initialize_state(grid_size, num_volunteers, num_victims)
    print("Initial State:", state)

    q_values = resq_rescue_scheduling(state, max_iterations=100)

    print("\nFinal Q-values:")
    for k, v in q_values.items():
        print(f"{k}: {v}")

Initial State: {'volunteers': {'volunteer_0': (24, 5), 'volunteer_1': (23, 0), 'volunteer_2': (15, 22), 'volunteer_3': (8, 15), 'volunteer_4': (8, 4)}, 'victims': {'victim_0': (1, 4), 'victim_1': (12, 11), 'victim_2': (5, 24), 'victim_3': (3, 3), 'victim_4': (8, 7), 'victim_5': (12, 17), 'victim_6': (23, 24), 'victim_7': (7, 21)}}

Final Q-values:
('{"victims": {"victim_0": [1, 4], "victim_1": [12, 11], "victim_2": [5, 24], "victim_3": [3, 3], "victim_4": [8, 7], "victim_5": [12, 17], "victim_6": [23, 24], "victim_7": [7, 21]}, "volunteers": {"volunteer_0": [24, 5], "volunteer_1": [23, 0], "volunteer_2": [15, 22], "volunteer_3": [8, 15], "volunteer_4": [8, 4]}}', 'up'): -6.95911207419159
('{"victims": {"victim_0": [1, 4], "victim_1": [12, 11], "victim_2": [5, 24], "victim_3": [3, 3], "victim_4": [8, 7], "victim_5": [12, 17], "victim_6": [23, 24], "victim_7": [7, 21]}, "volunteers": {"volunteer_0": [24, 5], "volunteer_1": [23, 0], "volunteer_2": [15, 22], "volunteer_3": [8, 15], "volunt